In [1]:
import pickle
import pandas as pd
import numpy as np
from tqdm import tqdm

In [2]:
train_df = pd.read_csv("../../input/feedback-prize-english-language-learning/train.csv")
train_df.head(2)

,text_id,full_text,cohesion,syntax,vocabulary,phraseology,grammar,conventions
0,0016926B079C,I think that students would benefit from learn...,3.5,3.5,3.0,3.0,4.0,3.0
1,0022683E9EA5,When a problem is a change you have to let it ...,2.5,2.5,3.0,2.0,2.0,2.5


In [3]:
model_path_list = [
    #('../../07_Embeddings/exp/result/07_v1_01/oof_df.csv',1), # svr
    
    #('../../14_Baseline4/exp/result/14_v1_01/oof_df.csv',1), # deberta-v3-base
    ('../../14_Baseline4/exp/result/14_v1_07/oof_df.csv',1), # deberta-v3-base, 10folds     
    #('../../14_Baseline4/exp/result/14_v1_08/oof_df.csv',1), # deberta-base
    #('../../14_Baseline4/exp/result/14_v1_09/oof_df.csv',1), # roberta-base
    ('../../14_Baseline4/exp/result/14_v1_10/oof_df.csv',1), # deberta-v3-large
    ('../../14_Baseline4/exp/result/14_v1_11/oof_df.csv',1), # xlm-roberta-base
    ('../../14_Baseline4/exp/result/14_v1_12/oof_df.csv',1), # deberta-large
    #('../../14_Baseline4/exp/result/14_v1_13/oof_df.csv',1), # roberta-large
    #('../../14_Baseline4/exp/result/14_v1_14/oof_df.csv',1), # xlm-roberta-large
#     ('../../14_Baseline4/exp/result/14_v1_15/oof_df.csv',1), # deberta-v2-xlarge
#     ('../../14_Baseline4/exp/result/14_v1_16/oof_df.csv',1), # deberta-xlarge
    
#     ('../../14_Baseline4/exp/result/14_v2_01/oof_df.csv',1), # deberta-v3-base, seed200
#     ('../../14_Baseline4/exp/result/14_v2_02/oof_df.csv',1), # deberta-v3-large, seed200
#     ('../../14_Baseline4/exp/result/14_v2_03/oof_df.csv',1), # deberta-base, seed200
    
    #('../../17_Pseudo3/exp/result/17_v1_01/oof_df.csv',1), # deberta-v3-base
    #('../../17_Pseudo3/exp/result/17_v1_02/oof_df.csv',1), # deberta-v3-base
    #('../../17_Pseudo3/exp/result/17_v1_03/oof_df.csv',1), # deberta-v3-large
    #('../../17_Pseudo3/exp/result/17_v1_04/oof_df.csv',1), # deberta-v3-base, seed200
    #('../../17_Pseudo3/exp/result/17_v1_05/oof_df.csv',1), # deberta-base
    #('../../17_Pseudo3/exp/result/17_v1_06/oof_df.csv',1), # deberta-large
    #('../../17_Pseudo3/exp/result/17_v1_07/oof_df.csv',1), # deberta-v3-small
    
    #('../../18_Pseudo4/exp/result/18_v1_01/oof_df.csv',1), # deberta-v3-base
    #('../../18_Pseudo4/exp/result/18_v1_02/oof_df.csv',1), # deberta-v3-large
    #('../../18_Pseudo4/exp/result/18_v1_03/oof_df.csv',1), # deberta-v3-small
    
    #('../../19_Distillation/exp/result/19_v1_01/oof_df.csv',1), # deberta-v3-base <- dbv3-l
    ('../../19_Distillation/exp/result/19_v1_02/oof_df.csv',1), # deberta-v3-base <- dbv3-xl
    ('../../19_Distillation/exp/result/19_v1_03/oof_df.csv',1), # deberta-v3-base <- db-xl
    #('../../19_Distillation/exp/result/19_v1_04/oof_df.csv',1), # deberta-v3-small <- dbv3-l
    #('../../19_Distillation/exp/result/19_v1_05/oof_df.csv',1), # deberta-v3-base <- db-l
    
    # single models
    ('../../20_Distillation2/exp/result/20_v2_01/oof_df.csv',1), # deberta-v3-base <- dbv3-b
    #('../../20_Distillation2/exp/result/20_v2_02/oof_df.csv',1), # deberta-v3-base <- dbv3-l
    #('../../20_Distillation2/exp/result/20_v2_03/oof_df.csv',1), # deberta-v3-large <- dbv3-l
    #('../../20_Distillation2/exp/result/20_v2_04/oof_df.csv',1), # deberta-base <- db-b
    #('../../20_Distillation2/exp/result/20_v2_05/oof_df.csv',1), # deberta-v3-base <- db-b
    #('../../20_Distillation2/exp/result/20_v2_06/oof_df.csv',1), # deberta-v3-base <- db-l
    ('../../20_Distillation2/exp/result/20_v2_07/oof_df.csv',1), # deberta-v3-base <- svr
]

oof_df_list = [
    pd.read_csv(model_path) for model_path,_ in model_path_list
]
weights_list = [
    w for _,w in model_path_list
]
weights_list = [w/sum(weights_list) for w in weights_list]

for oof_df in oof_df_list:
    oof_df = train_df[['text_id']].merge(oof_df, how='left', on='text_id')
    
num_models = len(model_path_list)

TARGET_COLS = ['cohesion','syntax','vocabulary','phraseology','grammar','conventions']

preds = 0
for oof_df, w in zip(oof_df_list, weights_list):
    preds += oof_df[TARGET_COLS].values * w

oofs = []
for oof_df in oof_df_list:
    oofs.append(oof_df[TARGET_COLS].values)

In [4]:
oof_df = oof_df_list[0].copy()
for i,col in enumerate(TARGET_COLS):
    oof_df[col] = preds[:,i]

In [5]:
import numpy as np

def calc_metric(pred, gt):
    '''
    pred : (num_data, num_labels)
    gt : (num_data, num_labels)
    '''
    score = np.sqrt(np.mean((pred - gt)**2, axis=0))
    score = score.mean()
    return score

In [6]:
score = calc_metric(pred=oof_df[TARGET_COLS].values, gt=train_df[TARGET_COLS].values)
print('CV={:.4f}'.format(score))

CV=0.4440


# Optimize class-wise

In [7]:
ensemble_predictions=np.stack(oofs)
ensemble_predictions.shape

(8, 3911, 6)

In [8]:
from skopt import gp_minimize

def run_optimize(i):
    target_col = TARGET_COLS[i]
    print("*"*50)
    print(target_col)
    print("*"*50)
    def ensemble_score(weights,return_pred=False):
        weights=np.array(weights)
        weights=weights.reshape(-1,1,1)/weights.sum()
        p=weights.reshape(-1,1,1)*ensemble_predictions[:,:,i,None]
        p=p.sum(0)
        score=calc_metric(p, train_df[[target_col]])
        if return_pred:
            return score,p
        else:
            return score   
    results = gp_minimize(ensemble_score, np.array([[0.1,1] for i in range(len(ensemble_predictions))]),
                          verbose=True,random_state=2022)
    best_weights=np.array(results['x'])/sum(results['x'])
    print(best_weights)
    # smaller is better for this metric
    score, ensemble_pred=ensemble_score(best_weights,True)
    print("score={:.4f}", score)
    return score, best_weights

In [9]:
scores = {}
best_weights = {}
for i,target_col in enumerate(TARGET_COLS):
    score, weights = run_optimize(i)
    scores[target_col] = score
    best_weights[target_col] = weights

**************************************************
cohesion
**************************************************
Iteration No: 1 started. Evaluating function at random point.
Iteration No: 1 ended. Evaluation done at random point.
Time taken: 0.0049
Function value obtained: 0.4739
Current minimum: 0.4739
Iteration No: 2 started. Evaluating function at random point.
Iteration No: 2 ended. Evaluation done at random point.
Time taken: 0.0019
Function value obtained: 0.4722
Current minimum: 0.4722
Iteration No: 3 started. Evaluating function at random point.
Iteration No: 3 ended. Evaluation done at random point.
Time taken: 0.0019
Function value obtained: 0.4730
Current minimum: 0.4722
Iteration No: 4 started. Evaluating function at random point.
Iteration No: 4 ended. Evaluation done at random point.
Time taken: 0.0019
Function value obtained: 0.4728
Current minimum: 0.4722
Iteration No: 5 started. Evaluating function at random point.
Iteration No: 5 ended. Evaluation done at random point.

Iteration No: 41 ended. Search finished for the next optimal point.
Time taken: 0.4196
Function value obtained: 0.4714
Current minimum: 0.4714
Iteration No: 42 started. Searching for the next optimal point.
Iteration No: 42 ended. Search finished for the next optimal point.
Time taken: 0.3972
Function value obtained: 0.4724
Current minimum: 0.4714
Iteration No: 43 started. Searching for the next optimal point.
Iteration No: 43 ended. Search finished for the next optimal point.
Time taken: 0.4141
Function value obtained: 0.4723
Current minimum: 0.4714
Iteration No: 44 started. Searching for the next optimal point.
Iteration No: 44 ended. Search finished for the next optimal point.
Time taken: 0.4378
Function value obtained: 0.4730
Current minimum: 0.4714
Iteration No: 45 started. Searching for the next optimal point.
Iteration No: 45 ended. Search finished for the next optimal point.
Time taken: 0.4476
Function value obtained: 0.4726
Current minimum: 0.4714
Iteration No: 46 started. Sea

Iteration No: 81 ended. Search finished for the next optimal point.
Time taken: 0.5431
Function value obtained: 0.4711
Current minimum: 0.4711
Iteration No: 82 started. Searching for the next optimal point.
Iteration No: 82 ended. Search finished for the next optimal point.
Time taken: 0.5380
Function value obtained: 0.4747
Current minimum: 0.4711
Iteration No: 83 started. Searching for the next optimal point.
Iteration No: 83 ended. Search finished for the next optimal point.
Time taken: 0.5566
Function value obtained: 0.4711
Current minimum: 0.4711
Iteration No: 84 started. Searching for the next optimal point.
Iteration No: 84 ended. Search finished for the next optimal point.
Time taken: 0.5442
Function value obtained: 0.4747
Current minimum: 0.4711
Iteration No: 85 started. Searching for the next optimal point.
Iteration No: 85 ended. Search finished for the next optimal point.
Time taken: 0.5546
Function value obtained: 0.4711
Current minimum: 0.4711
Iteration No: 86 started. Sea

Iteration No: 21 ended. Search finished for the next optimal point.
Time taken: 0.2621
Function value obtained: 0.4386
Current minimum: 0.4370
Iteration No: 22 started. Searching for the next optimal point.
Iteration No: 22 ended. Search finished for the next optimal point.
Time taken: 0.2641
Function value obtained: 0.4383
Current minimum: 0.4370
Iteration No: 23 started. Searching for the next optimal point.
Iteration No: 23 ended. Search finished for the next optimal point.
Time taken: 0.2959
Function value obtained: 0.4379
Current minimum: 0.4370
Iteration No: 24 started. Searching for the next optimal point.
Iteration No: 24 ended. Search finished for the next optimal point.
Time taken: 0.2801
Function value obtained: 0.4376
Current minimum: 0.4370
Iteration No: 25 started. Searching for the next optimal point.
Iteration No: 25 ended. Search finished for the next optimal point.
Time taken: 0.3097
Function value obtained: 0.4383
Current minimum: 0.4370
Iteration No: 26 started. Sea

Iteration No: 61 ended. Search finished for the next optimal point.
Time taken: 0.4808
Function value obtained: 0.4376
Current minimum: 0.4366
Iteration No: 62 started. Searching for the next optimal point.
Iteration No: 62 ended. Search finished for the next optimal point.
Time taken: 0.4641
Function value obtained: 0.4366
Current minimum: 0.4366
Iteration No: 63 started. Searching for the next optimal point.
Iteration No: 63 ended. Search finished for the next optimal point.
Time taken: 0.4531
Function value obtained: 0.4379
Current minimum: 0.4366
Iteration No: 64 started. Searching for the next optimal point.
Iteration No: 64 ended. Search finished for the next optimal point.
Time taken: 0.5050
Function value obtained: 0.4378
Current minimum: 0.4366
Iteration No: 65 started. Searching for the next optimal point.
Iteration No: 65 ended. Search finished for the next optimal point.
Time taken: 0.4563
Function value obtained: 0.4376
Current minimum: 0.4366
Iteration No: 66 started. Sea

...

Iteration No: 11 ended. Search finished for the next optimal point.
Time taken: 0.1869
Function value obtained: 0.4646
Current minimum: 0.4627
Iteration No: 12 started. Searching for the next optimal point.
Iteration No: 12 ended. Search finished for the next optimal point.
Time taken: 0.3202
Function value obtained: 0.4653
Current minimum: 0.4627
Iteration No: 13 started. Searching for the next optimal point.
Iteration No: 13 ended. Search finished for the next optimal point.
Time taken: 0.3339
Function value obtained: 0.4632
Current minimum: 0.4627
Iteration No: 14 started. Searching for the next optimal point.
Iteration No: 14 ended. Search finished for the next optimal point.
Time taken: 0.3702
Function value obtained: 0.4634
Current minimum: 0.4627
Iteration No: 15 started. Searching for the next optimal point.
Iteration No: 15 ended. Search finished for the next optimal point.
Time taken: 0.3501
Function value obtained: 0.4643
Current minimum: 0.4627
Iteration No: 16 started. Sea

Iteration No: 51 ended. Search finished for the next optimal point.
Time taken: 0.4629
Function value obtained: 0.4633
Current minimum: 0.4623
Iteration No: 52 started. Searching for the next optimal point.
Iteration No: 52 ended. Search finished for the next optimal point.
Time taken: 0.4799
Function value obtained: 0.4630
Current minimum: 0.4623
Iteration No: 53 started. Searching for the next optimal point.
Iteration No: 53 ended. Search finished for the next optimal point.
Time taken: 0.4858
Function value obtained: 0.4637
Current minimum: 0.4623
Iteration No: 54 started. Searching for the next optimal point.
Iteration No: 54 ended. Search finished for the next optimal point.
Time taken: 0.5026
Function value obtained: 0.4623
Current minimum: 0.4623
Iteration No: 55 started. Searching for the next optimal point.
Iteration No: 55 ended. Search finished for the next optimal point.
Time taken: 0.5506
Function value obtained: 0.4633
Current minimum: 0.4623
Iteration No: 56 started. Sea

Iteration No: 91 ended. Search finished for the next optimal point.
Time taken: 0.7239
Function value obtained: 0.4622
Current minimum: 0.4622
Iteration No: 92 started. Searching for the next optimal point.
Iteration No: 92 ended. Search finished for the next optimal point.
Time taken: 0.7440
Function value obtained: 0.4637
Current minimum: 0.4622
Iteration No: 93 started. Searching for the next optimal point.
Iteration No: 93 ended. Search finished for the next optimal point.
Time taken: 0.7720
Function value obtained: 0.4631
Current minimum: 0.4622
Iteration No: 94 started. Searching for the next optimal point.
Iteration No: 94 ended. Search finished for the next optimal point.
Time taken: 0.7420
Function value obtained: 0.4654
Current minimum: 0.4622
Iteration No: 95 started. Searching for the next optimal point.
Iteration No: 95 ended. Search finished for the next optimal point.
Time taken: 0.7671
Function value obtained: 0.4645
Current minimum: 0.4622
Iteration No: 96 started. Sea

Iteration No: 31 ended. Search finished for the next optimal point.
Time taken: 0.3553
Function value obtained: 0.4375
Current minimum: 0.4369
Iteration No: 32 started. Searching for the next optimal point.
Iteration No: 32 ended. Search finished for the next optimal point.
Time taken: 0.2943
Function value obtained: 0.4373
Current minimum: 0.4369
Iteration No: 33 started. Searching for the next optimal point.
Iteration No: 33 ended. Search finished for the next optimal point.
Time taken: 0.3665
Function value obtained: 0.4370
Current minimum: 0.4369
Iteration No: 34 started. Searching for the next optimal point.
Iteration No: 34 ended. Search finished for the next optimal point.
Time taken: 0.3500
Function value obtained: 0.4374
Current minimum: 0.4369
Iteration No: 35 started. Searching for the next optimal point.
Iteration No: 35 ended. Search finished for the next optimal point.
Time taken: 0.3488
Function value obtained: 0.4374
Current minimum: 0.4369
Iteration No: 36 started. Sea

Iteration No: 71 ended. Search finished for the next optimal point.
Time taken: 0.5035
Function value obtained: 0.4373
Current minimum: 0.4368
Iteration No: 72 started. Searching for the next optimal point.
Iteration No: 72 ended. Search finished for the next optimal point.
Time taken: 0.5223
Function value obtained: 0.4380
Current minimum: 0.4368
Iteration No: 73 started. Searching for the next optimal point.
Iteration No: 73 ended. Search finished for the next optimal point.
Time taken: 0.5565
Function value obtained: 0.4368
Current minimum: 0.4368
Iteration No: 74 started. Searching for the next optimal point.
Iteration No: 74 ended. Search finished for the next optimal point.
Time taken: 0.5507
Function value obtained: 0.4379
Current minimum: 0.4368
Iteration No: 75 started. Searching for the next optimal point.
Iteration No: 75 ended. Search finished for the next optimal point.
Time taken: 0.5405
Function value obtained: 0.4373
Current minimum: 0.4368
Iteration No: 76 started. Sea

In [10]:
best_weights

{'cohesion': array([0.28290196, 0.14574802, 0.0282902 , 0.26929603, 0.13147485,
        0.08570855, 0.0282902 , 0.0282902 ]),
 'syntax': array([0.23935735, 0.20296328, 0.0633982 , 0.25199949, 0.16668183,
        0.02519995, 0.02519995, 0.02519995]),
 'vocabulary': array([0.23548926, 0.30257562, 0.12813107, 0.14224365, 0.03025756,
        0.03025756, 0.10078772, 0.03025756]),
 'phraseology': array([0.22226812, 0.20820921, 0.04830378, 0.1609494 , 0.14163953,
        0.06356874, 0.13102727, 0.02403396]),
 'grammar': array([0.23180728, 0.39263333, 0.04619996, 0.04619996, 0.14455961,
        0.04619996, 0.04619996, 0.04619996]),
 'conventions': array([0.23424934, 0.16494992, 0.08039204, 0.15263777, 0.15931657,
        0.16160449, 0.02342493, 0.02342493])}

In [11]:
scores

{'cohesion': 0.4711007963092495,
 'syntax': 0.4365577352836148,
 'vocabulary': 0.4048063768778469,
 'phraseology': 0.44588024557667505,
 'grammar': 0.46215805084008765,
 'conventions': 0.43675676709874467}

In [12]:
CV = sum(scores.values()) / len(scores)
print("CV={:.4f}".format(CV))

CV=0.4429


# Error Analysis - Check Corr

In [13]:
cols = ['cohesion','syntax','vocabulary','phraseology','grammar','conventions']
train_df[cols].corr()

,cohesion,syntax,vocabulary,phraseology,grammar,conventions
cohesion,1.000000,0.695459,0.666151,0.690058,0.638689,0.666151
syntax,0.695459,1.000000,0.680562,0.725467,0.709525,0.700025
vocabulary,0.666151,0.680562,1.000000,0.735261,0.654852,0.664292
phraseology,0.690058,0.725467,0.735261,1.000000,0.719746,0.666842
grammar,0.638689,0.709525,0.654852,0.719746,1.000000,0.673301
conventions,0.666151,0.700025,0.664292,0.666842,0.673301,1.000000


In [14]:
cols = ['cohesion','syntax','vocabulary','phraseology','grammar','conventions']
oof_df[cols].corr()

,cohesion,syntax,vocabulary,phraseology,grammar,conventions
cohesion,1.000000,0.957022,0.929827,0.922988,0.843317,0.901071
syntax,0.957022,1.000000,0.948949,0.967860,0.928577,0.935624
vocabulary,0.929827,0.948949,1.000000,0.965449,0.890111,0.901799
phraseology,0.922988,0.967860,0.965449,1.000000,0.952139,0.893484
grammar,0.843317,0.928577,0.890111,0.952139,1.000000,0.867320
conventions,0.901071,0.935624,0.901799,0.893484,0.867320,1.000000
